# Exploring Random Forests

In [4]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.datasets import load_boston, load_iris, load_wine, load_digits, \
                             load_breast_cancer, load_diabetes

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

from rfpimp import *

from distutils.version import LooseVersion
if LooseVersion(sklearn.__version__) >= LooseVersion("0.24"):
    # In sklearn version 0.24, forest module changed to be private.
    from sklearn.ensemble._forest import _generate_unsampled_indices
    from sklearn.ensemble import _forest as forest
else:
    # Before sklearn version 0.24, forest was public, supporting this.
    from sklearn.ensemble.forest import _generate_unsampled_indices
    from sklearn.ensemble import forest

from sklearn import tree
from dtreeviz.trees import *

In [5]:
def rent(n=None, bootstrap=False):
    df_rent = pd.read_csv("data/rent-ideal.csv")
    if n is None:
        n = len(df_rent)
    df_rent = df_rent.sample(n, replace=bootstrap)
    X = df_rent[['bedrooms','bathrooms','latitude','longitude']]
    y = df_rent['price']
    return X, y

def boston():
    boston = load_boston()
    X = boston.data
    y = boston.target
    features = boston.feature_names
    df = pd.DataFrame(data=X,columns=features)
    df['y'] = y
    return df

## Set up

Get the `rent-ideal.csv`  data file from canvas and store in the data directory underneath your notebook  directory.

In [6]:
X, y = rent()
X.head(3)

,bedrooms,bathrooms,latitude,longitude
23112,2,1.0,40.7961,-73.9685
31274,2,1.5,40.7550,-73.9636
22091,1,1.0,40.7982,-73.9686


In [7]:
X.shape

(48300, 4)

## Train random forests of different sizes

As we increase the number of trees in the forest, we initially see model bias going down. It will asymptotically approach some minimum error on the testing set.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

Here's how to train a random forest  that has a single tree:

In [13]:
rf = RandomForestRegressor(n_estimators=1)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1)

**Task**: Compute the MAE for the training and the testing set, printing them out.

In [ ]:
mae_train = mean_absolute_error(...)
mae = mean_absolute_error(...)
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

<details>
<summary>Solution</summary>
<pre>
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
</pre>
</details>

**Task**: Run the training and testing cycle several times to see the variance: the test scores bounce around a lot.

**Task**: Increase the number of trees (`n_estimators`) to 2, retrain, and print out the results.

In [ ]:
rf = ...
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

<details>
<summary>Solution</summary>
<pre>
rf = RandomForestRegressor(n_estimators=2)
rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")
</pre>
</details>

You should notice the both test MAE scores going down and bouncing around less from run to run.

**Q.**  Why does the MAE score go down?

<details>
<summary>Solution</summary>
    With 2 trees, the chances are that the random forest will have seen (trained on) more of the original training set, despite bootstrapping.
</details>

**Task**: Increase the number of trees (`n_estimators`) to 10, retrain, and print out the results.

In [ ]:
rf = ...
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

<details>
<summary>Solution</summary>
<pre>
rf = RandomForestRegressor(n_estimators=10)
rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")
</pre>
</details>

**Q.**  What you notice about the MAE scores?

<details>
<summary>Solution</summary>
They are getting smaller.
</details>

**Q.**  After running several times, what else do you notice?

<details>
<summary>Solution</summary>
    With 10 trees, the prediction from run to run varies a lot less. We have reduced variance,  improving generality.
</details>

**Task**: Increase the number of trees (`n_estimators`) to 200, retrain, and print out the results.

In [ ]:
rf = ...
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

<details>
<summary>Solution</summary>
<pre>
rf = RandomForestRegressor(n_estimators=200)
%time rf.fit(X_train, y_train) # how long does this take?
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")
</pre>
</details>

**Q.**  What you notice about the MAE scores from a single run?

<details>
<summary>Solution</summary>
They are a bit smaller, but not by much.
</details>

**Task**: Notice that it took a long time to train, about 10 seconds.  Do the exact same thing again but this time use `n_jobs=-1` as an argument to the `RandomForestRegressor` constructor.

This tells the library to use all processing cores available on the computer processor. As long as the data is not too huge (because it must pass it around), it often goes much faster using this argument. It should take less than two seconds.

In [ ]:
rf = ...
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

<details>
<summary>Solution</summary>
<pre>
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")
</pre>
</details>

**Q.**  What you notice about the MAE scores from SEVERAL runs?

<details>
<summary>Solution</summary>
The error variance across runs is even lower (tighter).
</details>

## Examining model size and complexity

The structure of a tree is affected by a number of hyper parameters, not just the data. Goal in the section is to see the effect of altering the number of samples per leaf and the maximum number of candidate features per split. Let's start out with a handy function that uses some  support code from rfpimp to examine tree size and depth:

In [ ]:
def showsize(ntrees, max_features=1.0, min_samples_leaf=1):
    rf = RandomForestRegressor(n_estimators=ntrees,
                               max_features=max_features,
                               min_samples_leaf=min_samples_leaf,
                               n_jobs=-1)
    rf.fit(X_train, y_train)
    n = rfnnodes(rf)                # from rfpimp
    h = np.median(rfmaxdepths(rf))  # rfmaxdepths from rfpimp
    mae_train = mean_absolute_error(y_train, rf.predict(X_train))
    mae = mean_absolute_error(y_test, rf.predict(X_test))
    print(f"MAE train {mae_train:6.1f}$, test {mae:6.1f}$ using {n:9,d} tree nodes with {h:2.0f} median tree height")

### Effect of number of trees

For a single tree, we see about 21,000 nodes and a tree height of around 35:

In [ ]:
showsize(ntrees=1)

**Task**: Look at the metrics for 2 trees and then 100 trees.

<details>
<summary>Solution</summary>
<pre>
showsize(ntrees=2)
showsize(ntrees=100)
</pre>
</details>

**Q.** Why does the median height of a tree stay the same when we increase the number of trees?

<details>
<summary>Solution</summary>
While the number of nodes increases with the number of trees, the height of any individual tree will stay the same because we have not fundamentally changed how it is constructing a single tree.
</details>

### Effect of increasing min samples / leaf

**Task**: Loop around a call to `showsize()` with 10 trees and min_samples_leaf=1..10 

In [ ]:
for i in range(...):
    print(f"{i:2d} ",end='')
    showsize(...)

<details>
<summary>Solution</summary>
<pre>
for i in range(1,10+1):
    showsize(ntrees=10, min_samples_leaf=i)
</pre>
</details>

**Q.** Why do the median height of a tree and number of total nodes decrease as we increase the number of samples per leaf?

<details>
<summary>Solution</summary>
Because  when the sample size gets down to `min_samples_leaf`, splitting stops, which prevents the tree from getting taller. It also restricts how many nodes total get created for the tree.
</details> 

**Q.**  Why does the MAE error increase?

<details>
<summary>Solution</summary>
If we include more observations in a single leaf, then the average is taken over more samples. That average is a more general prediction but less accurate.
</details> 

It's pretty clear from that print out that `min_samples_leaf=1` is the best choice because it gives the minimum validation error.

### Effect of reducing max_features (rent data)

**Task:** Do another loop from `max_features` = 4 down to 1, with 1 sample per leaf. (There are 4 total features.)

In [ ]:
p = X_train.shape[1]
for i in range(...):
    print(f"{i:2d} ",end='')
    showsize(ntrees=10, ...)

<details>
<summary>Solution</summary>
<pre>
p = X_train.shape[1]
for i in range(p,0,-1):
    print(f"{i:2d} ",end='')
    showsize(ntrees=10, max_features=i)
</pre>
</details>

For this data set,  changing the available candidate features that each split does not seem to be important as the validation error does not change, nor does the height of the trees.

### Examine effects of hyper parameters on Boston data set

In [ ]:
df_boston = boston()
df_boston.head(3)

In [ ]:
X, y = df_boston.drop('y', axis=1), df_boston['y']
y *= 1000 # y is "Median value of owner-occupied homes in $1000's" so multiply by 1000

# reproducible 20% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

Let's run the metric `showsize()` function to see how many trees we should use:

In [ ]:
for i in [1,5,30,50,100,150,300]:
    print(f"{i:3d} trees: ", end='')
    showsize(ntrees=i)

Seems like the sweet spot on the validation error is probably 150 trees as it gets a low validation error and has a fairly small set of trees.

Check the effect of increasing the minimum samples per leaf from 1 to 10 as we did before.

In [ ]:
for i in range(1,10+1):
    print(f"{i:2d} ",end='')
    showsize(ntrees=150, min_samples_leaf=i)

The training error goes up dramatically but the validation error doesn't get too much worse.  

**Q.**  Which min samples per leaf would you choose?

<details>
<summary>Solution</summary>
    After running a few times, it seems that using <tt>min_samples_leaf</tt>=1 or 2 is best for the validation error. But, keep in mind that this data set is pretty small and so our error values will change quite a bit depending on the sample we get for the test set.
</details> 

Run a loop from the maximum number of features down to 1 for `max_features` to see the effects.

In [ ]:
p = X_train.shape[1]
for i in range(p,0,-1):
    print(f"{i:2d} ",end='')
    showsize(ntrees=150, max_features=i, min_samples_leaf=3)

**Q.** Which max features would you choose?

<details>
<summary>Solution</summary>
    After running a few times, it seems that using <tt>max_features</tt>=7 or 13 gets best validation error, but again it depends on the randomness of the tree construction and results will vary across runs.
</details> 

Here's what the final model would look like:

In [ ]:
showsize(ntrees=150, max_features=13, min_samples_leaf=1)

## RF prediction confidence

A random forest is a collection of decision trees, each of which contributes a prediction. The forest averages those predictions to provide the overall prediction (or takes most common vote for classification). Let's dig inside the random forest to get the individual trees out and ask them what their predictions are.

**Task**: Train a random forest with 10 trees on `X_train`, `y_train`.  Use `for t in rf.estimators_` to iterate through the trees making predictions with `t` not `rf`. Print out the usual MAE scores for each tree predictor.

In [ ]:
rf = RandomForestRegressor(n_estimators=10, n_jobs=-1)
rf.fit(X_train, y_train)

for t in ...:
    mae_train = ...
    mae = ...
    print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

<details>
<summary>Solution</summary>
<pre>
rf = RandomForestRegressor(n_estimators=10, n_jobs=-1)
rf.fit(X_train, y_train)

for t in rf.estimators_:
    mae_train = mean_absolute_error(y_train, t.predict(X_train))
    mae = mean_absolute_error(y_test, t.predict(X_test))
    print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")
</pre>
</details>

Notice that it bounces around quite a bit. 

**Task**: Select one of the `X_test` rows and print out the addicted rent price.

In [ ]:
x = ... # pick single test case
x = x.values.reshape(1,-1) # Needs to be a one-row matrix

print(f"{x} => {rf.predict(x)}$")

<details>
<summary>Solution</summary>
<pre>
x = X_test.iloc[3,:] # pick single test case
x = x.values.reshape(1,-1)
print(f"{x} => {rf.predict(x)}$")
</pre>
</details>

**Task**: Now let's see how the forest came to that conclusion. Compute the average of the predictions obtained from every tree.  

Compare that to the prediction obtained directly from the random forest (`rf.predict(X_test)`). They should be the same.

In [ ]:
y_pred = ...
print(f"{x} => {y_pred}$")

<details>
<summary>Solution</summary>
<pre>
y_pred = np.mean([t.predict(x) for t in rf.estimators_])
print(f"{x} => {y_pred}$")
</pre>
</details>

**Task**: Compute the standard deviation of the tree estimates and print that out.

<details>
<summary>Solution</summary>
<pre>
np.std([t.predict(x) for t in rf.estimators_])
</pre>
</details>

The lower the standard deviation, the more tightly grouped the predictions were, which means we should have more confidence in our answer. 

Different records will often have different standard deviations, which means we could have different levels of confidence in the various answers. This might be helpful to a bank for example that wanted to not only predict whether to give loans, but how confident the model was.

## Altering bootstrap size

**This no longer works with latest versions of scikit-learn... and the feature is not yet implemented by them* See [related github issue](https://github.com/scikit-learn/scikit-learn/issues/11993).  Ah [this new features](https://github.com/scikit-learn/scikit-learn/pull/14682) covers it for trees. "Adds a max_samples kwarg to forest ensembles that limits the size of the bootstrap samples used to train each estimator."

In [15]:
X, y = rent()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

**Task**: There are about 38,000 training records, change that to 19,000 and check the accuracy again.

In [16]:
rf = RandomForestRegressor(n_estimators=200) # don't compute in parallel so we can see timing
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

CPU times: user 8.64 s, sys: 235 ms, total: 8.88 s
Wall time: 8.95 s
MAE train 184.2$, test 291.1$


In [18]:
rf = RandomForestRegressor(n_estimators=200, max_samples=1/2)
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

CPU times: user 5.75 s, sys: 122 ms, total: 5.88 s
Wall time: 5.96 s
MAE train 226.7$, test 307.1$


It's a bit less accurate, but it's faster.

**Q.**  Why is it less accurate?

<details>
<summary>Solution</summary>
Each tree is seeing less of the data set during training.
</details>

**Task**: Turn off bootstrapping by adding `bootstrap=False` to the constructor of the model. This means that it will subsample rather than bootstrap. Remember that bootstrapping gets about two thirds of the data because of replacement.

In [ ]:
rf = ...
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

<details>
<summary>Solution</summary>
<pre>
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1, bootstrap=False)
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")
</pre>
</details>

That brings the accuracy back up a little bit for the test set but very much so for the training MAE score.

**Task**: Drop that size to one third of the training records then retrain and test.

In [ ]:
jeremy_trick_RF_sample_size(round(len(X_train)/3))

In [ ]:
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

Mine is twice as fast as the full bootstrap but continues to have very tight variance because of the number of trees. The accuracy is lower, however, about what we get for the  usual random forest with two trees.